<a href="https://colab.research.google.com/github/santoshrsarangi/tensorflow/blob/main/NLP/RNN_IMDB_Movie_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  8921k      0  0:00:09  0:00:09 --:--:-- 16.1M


In [2]:
!tar -xf aclImdb_v1.tar.gz

In [3]:
!ls -l

total 82164
drwxr-xr-x 4 7297 1000     4096 Jun 26  2011 aclImdb
-rw-r--r-- 1 root root 84125825 Jun  4 18:13 aclImdb_v1.tar.gz
drwxr-xr-x 1 root root     4096 Jun  1 22:51 sample_data


In [4]:
!rm -r aclImdb/train/unsup

In [5]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [6]:
import os, pathlib, shutil, random

In [7]:
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

In [8]:
for category in ("pos", "neg"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files)
  num_of_val_files = int(len(files) * 0.2)

  val_files = files[-num_of_val_files:]

  for fname in val_files:
    shutil.move(train_dir / category / fname, val_dir / category / fname)

In [9]:
import tensorflow as tf
from tensorflow import keras

In [10]:
batch_size = 32

In [11]:
train_ds = keras.utils.text_dataset_from_directory("aclImdb/train", batch_size = batch_size)

Found 20000 files belonging to 2 classes.


In [12]:
val_ds = keras.utils.text_dataset_from_directory("aclImdb/val", batch_size=batch_size)

Found 5000 files belonging to 2 classes.


In [13]:
test_ds = keras.utils.text_dataset_from_directory("aclImdb/test", batch_size = batch_size)

Found 25000 files belonging to 2 classes.


In [14]:
for inputs, targets in train_ds:
  print(inputs[0])
  print(targets[0])
  print(inputs.shape)
  print(targets.shape)
  print(inputs.dtype)
  print(targets.dtype)
  break

tf.Tensor(b"Warning! Spoilers ahead!<br /><br /> SPOILERS<br /><br /> I've seen movie in German, so it might be, that I missed some clues.<br /><br /> Despite some weakness in the plot, it's a movie that came through to me. I liked especially Lexa Doig's acting. Sometimes I got impression, that she *is* Camille. But I can't stop wondering, what happened at the end with Bob, Cassie and baby. I belive, she, after initially being set on Bob, eventually ended up loving him and regretting what happened with his brother and being forced to lie to him. Otherwise it's a bit strange, that she would carry his baby and love it. It's up to viewer to decide - and I don't like such endings. Dean Cain was as good as ever, Eric Roberts .. well, I've seen him better but also worse.<br /><br /> I believe that the film is more an analysis of human relations and reacting in unexpected situations than a crime story.<br /><br /> Bottom line is, I liked it very much.", shape=(), dtype=string)
tf.Tensor(1, sh

In [15]:
from keras.layers import TextVectorization

In [48]:
#test the model accuracy with different values of ngram. Does the accuracy improve? why?
text_vectorization = TextVectorization(output_mode="multi_hot", max_tokens=20000, ngrams=3)

In [49]:
text_ds = train_ds.map(lambda x, y: x)

In [50]:
text_vectorization.adapt(text_ds)

In [51]:
ds = text_vectorization.get_vocabulary()

In [52]:
print(ds)

['[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i', 'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but', 'of the', 'film', 'on', 'not', 'you', 'his', 'are', 'have', 'he', 'be', 'one', 'in the', 'its', 'at', 'all', 'by', 'an', 'they', 'from', 'who', 'so', 'like', 'her', 'or', 'just', 'about', 'has', 'if', 'out', 'some', 'this movie', 'there', 'what', 'good', 'more', 'when', 'very', 'and the', 'is a', 'she', 'my', 'even', 'no', 'the film', 'up', 'would', 'to the', 'which', 'time', 'only', 'to be', 'really', 'story', 'their', 'were', 'had', 'see', 'can', 'the movie', 'me', 'this film', 'it is', 'than', 'we', 'much', 'this is', 'well', 'been', 'get', 'also', 'will', 'people', 'because', 'into', 'do', 'great', 'other', 'on the', 'bad', 'first', 'in a', 'how', 'most', 'him', 'it was', 'dont', 'for the', 'one of', 'made', 'then', 'of a', 'films', 'movies', 'with the', 'could', 'make', 'way', 'them', 'any', 'after', 'br the', 'too', 'think', 'characters', 'is the', 'as

In [37]:
len(ds)

20000

In [53]:
binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [54]:
binary_1gram_val_ds = val_ds.map(lambda x, y : (text_vectorization(x), y), num_parallel_calls=4)

In [55]:
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [41]:
for inputs, targets in binary_1gram_train_ds:
  print(inputs[0])
  print(targets[0])
  break

tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
tf.Tensor(0, shape=(), dtype=int32)


In [56]:
def get_model(max_tokens=20000, hidden_dims=16):

  inputs = keras.Input(shape=(max_tokens,))
  x = keras.layers.Dense(hidden_dims, activation="relu")(inputs)
  x = keras.layers.Dropout(0.5)(x)
  outputs = keras.layers.Dense(1, activation="sigmoid")(x)

  model = keras.Model(inputs, outputs)
  model.compile(optimizer="rmsprop",
                loss="binary_crossentropy",
                metrics=["accuracy"])
  
  return model


In [57]:
model = get_model()

In [58]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_4 (Dense)             (None, 16)                320016    
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


In [59]:
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_3gram.keras", save_best_only=True)
]

In [60]:
history = model.fit(binary_1gram_train_ds.cache(), 
                    validation_data = binary_1gram_val_ds.cache(),
                    callbacks=callbacks,
                    epochs=10)

Epoch 1/10
625/625 [==============================] - 9s 13ms/step - loss: 0.3745 - accuracy: 0.8453 - val_loss: 0.2645 - val_accuracy: 0.8952
Epoch 2/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2420 - accuracy: 0.9137 - val_loss: 0.2751 - val_accuracy: 0.9002
Epoch 3/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2044 - accuracy: 0.9315 - val_loss: 0.2931 - val_accuracy: 0.8950
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.1852 - accuracy: 0.9434 - val_loss: 0.3056 - val_accuracy: 0.9004
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.1779 - accuracy: 0.9481 - val_loss: 0.3227 - val_accuracy: 0.9010
Epoch 6/10
625/625 [==============================] - 3s 4ms/step - loss: 0.1679 - accuracy: 0.9507 - val_loss: 0.3413 - val_accuracy: 0.8994
Epoch 7/10
625/625 [==============================] - 2s 4ms/step - loss: 0.1586 - accuracy: 0.9535 - val_loss: 0.3630 - val_accuracy: 0.8984
Epoch

In [31]:
#result with 20000 dictionary words
model.evaluate(binary_1gram_test_ds)

782/782 [==============================] - 5s 6ms/step - loss: 0.3700 - accuracy: 0.8821


[0.3700084686279297, 0.8821200132369995]

In [47]:
#result with 2 ngram and 20000 dictionary word 
model.evaluate(binary_1gram_test_ds)

782/782 [==============================] - 4s 5ms/step - loss: 0.3748 - accuracy: 0.8890


[0.37480250000953674, 0.8890399932861328]

In [61]:
#result with 3 ngrams and 20000 dictionary word
model.evaluate(binary_1gram_test_ds)

782/782 [==============================] - 5s 6ms/step - loss: 0.3864 - accuracy: 0.8895


[0.3863873779773712, 0.889519989490509]